In [ ]:
# import os
import numpy as np
import os
import pylab as plt
import soundfile as sf
import pickle
from spikes import *
from brian import *
from brian.hears import *

f_res = 128 # frequency resolution value (number of neurons to model)
t_res = 102.4 # time reesolution value (points per second)

# frequency range to process
fr_min = 20 * Hz
fr_max = 5.5 * kHz

def find_nearest_idx(array, value):
    return np.abs(array - value).argmin()

def save_obj(obj, path):
    head, _, _ = path.partition('.')
    with open(head + '.pickle', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

def wav_to_spikes_pickle(filename, cfmin=50 * Hz, cfmax=11 * kHz,
                      cfN=128, threshold=0.5, refractory=5 * ms, threads=4,
                      eqs = '''
                      dv/dt = (I-v)/(2*ms): 1
                      I : 1
                      '''):
    # Inner hair cell model
    cf = erbspace(cfmin, cfmax, cfN)

    data, srate = sf.read(filename)
    sound = Sound(data, srate*Hz)
    if sound.shape[1] > 1:
        sound = sound.left

    gfb = Gammatone(sound, cf)

    ihc = FunctionFilterbank(gfb, lambda x: 2.5 * clip(x, 0, Inf) ** (1.0 / 5.0))
    # Leaky integrate-and-fire model with refractoriness
    G = FilterbankGroup(ihc, 'I', eqs, reset=0, threshold=threshold,
                        refractory=refractory)
    # Run, and raster plot of the spikes
    M = SpikeMonitor(G)
    # Seems threads don't work at all
    run(sound.duration, threads=8)
    # Save raw spikes to pickle file
    save_obj(M.spikes, filename)

    
def dir_to_spikes_pickle(path, filetype, cfN=128, threads=4):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file[-len(filetype):] == filetype:
                filename = root + '/' + file
                wav_to_spikes_pickle(filename, cfN=cfN, threads=threads)
                print filename
                
        
def dir_pickle_to_mat(path, f_res, t_res):
    data = []
    target = []
    for root, dirs, files in os.walk(path):
        for file in files:
            head, sep, tail = file.partition('.')
            if tail == 'pickle':
                filename = root+'/'+file
                spikes = np.array(load_obj(filename))
                # I use here 5 sec time length for whis dataset only
                # In general case better to use datafeeder 
                # with general length within 1 batch instead of whole dataset
                tmat = np.zeros((int(5*t_res), f_res), dtype='int8')
                if (spikes.any()):
                    t_scale = np.arange(0.0, spikes.T[1].max(), 1.0/t_res)
                    for spike in spikes:
                        idx = find_nearest_idx(t_scale, spike[1])
                        if (idx < tmat.shape[0]):
                            tmat[idx][int(spike[0])] = 1
                    
                    plt.imsave(filename[:-len(tail)] + 'jpg', tmat.T, cmap='Greys', origin='lower')
                    target.append(int(root.split()[0][-3:])-1)
                    data.append(tmat)
                    print(spikes.shape[0]/float(tmat.sum()), spikes.T[1].max(), filename[:-len(tail)])
    save = {'dataset' : np.array(data), 'labels' : np.array(target)}
    save_obj(save, 'esc10.pickle')

In [ ]:
dir_to_spikes_pickle("/media/sprocketus/fast/speech_corpus/ESC-10-master/004 - Baby cry/", filetype='ogg', threads=8)
dir_to_spikes_pickle("/media/sprocketus/fast/speech_corpus/ESC-10-master/006 - Person sneeze/", filetype='ogg', threads=8)
dir_to_spikes_pickle("/media/sprocketus/fast/speech_corpus/ESC-10-master/008 - Chainsaw/", filetype='ogg', threads=8)
dir_to_spikes_pickle("/media/sprocketus/fast/speech_corpus/ESC-10-master/009 - Rooster/", filetype='ogg', threads=8)

In [ ]:
dir_pickle_to_mat('/media/sprocketus/fast/speech_corpus/ESC-10-master/', f_res, t_res)